# Using tools for a price flight assistant chatbot

**Tools** in context of LLMs are external for LLM functions, that LLM can use for defined tasks. In our case tools are 2 python functions: _get_airport_ and _get_base_price_. With tools LLM doesn't have to do everything "in its head", but can integrate with classic deterministic programs. Exists also MCP (Model Context Protocol) client-server protocol for AI agents to dynamically discover, connect to, and interact with external tools and data sources in a standardized, secure way. It is way much more complicated that standard function calling (like in this example), and only should be used either when we don't know upfront which tools are available, or we need authentication before calling those tools.


In this example we just use a **static descriptor**, where we mention function name, what the tool can do and what are the input parameters. Format of the descriptor (constant TOOLS) is standard for most of LLM (at least for Ollama and OpenAI it is the same).


In this example I had to include a function is_greeting to make LLM **not** to use tools, otherwise it wants to use them on every single message.


Example below is a chatbot with Gradio UI, and uses 2 python functions (tools):
- get_airport retrieves a 3-letter airport code by city name
- get_base_price calculates the price depending on the airport code and date (summer and weekends are more expensive)


**Chat** looks like this:

![image](chat.png)

Every call to tools has a print statement, so the **log** looks like this:

![image](log.png)

In [ ]:
import json, ollama, gradio as gr
from datetime import datetime

MODEL = "llama3.2:3b"

SYSTEM_PROMPT = """
You are a helpful assistant for FlyCheap airline.
All flights depart from London Heathrow (LHR) unless the user says otherwise.
Give **short, courteous answers – maximum 1 sentence**.
**Always be accurate. If you don't know the answer, say so.**

YOU MUST USE TOOLS:
- Use `get_airport` **only** when the user asks for an airport code.
- Use `get_base_price` **only** when the user asks for a price AND mentions a destination (city or airport) AND a date.
  Example: "price to Berlin on 2025-11-15" → call get_base_price(airport="BER", date="2025-11-15")

If you cannot call a tool (e.g. missing date), reply:
"I need both a destination and a date to give you a price."

For greetings, just say hello back.
"""

# ------------------------------------------------------------------
#  TOOL DEFINITIONS (always passed to Ollama)
# ------------------------------------------------------------------
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_airport",
            "description": "Return the IATA code for a city/airport name.",
            "parameters": {
                "type": "object",
                "properties": {"city": {"type": "string"}},
                "required": ["city"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_base_price",
            "description": "Return the base return price from LHR to the destination airport on the given date.",
            "parameters": {
                "type": "object",
                "properties": {
                    "airport": {"type": "string", "description": "3-letter IATA code"},
                    "date": {"type": "string", "format": "date", "description": "YYYY-MM-DD"}
                },
                "required": ["airport", "date"]
            }
        }
    }
]


# ------------------------------------------------------------------
#  TOOL IMPLEMENTATIONS
# ------------------------------------------------------------------
def get_airport(city: str) -> str:
    print(f"[TOOL] get_airport('{city}')")
    mapping = {
        "London Heathrow": "LHR", "London Gatwick": "LGW", "London Stansted": "STN",
        "London Luton": "LTN", "Manchester": "MAN", "Edinburgh": "EDI", "Bristol": "BRS",
        "Paris Charles de Gaulle": "CDG", "Paris Orly": "ORY", "Nice Côte d'Azur": "NCE",
        "Lyon Saint-Exupéry": "LYS", "Marseille Provence": "MRS", "Toulouse Blagnac": "TLS",
        "Frankfurt": "FRA", "Munich": "MUC", "Düsseldorf": "DUS", "Hamburg": "HAM",
        "Berlin (Tegel)": "TXL", "Berlin Brandenburg": "BER",
        "Barcelona": "BCN", "Madrid": "MAD", "Málaga": "AGP",
        "Palma de Mallorca": "PMI", "Alicante": "ALC", "Valencia": "VLC", "Seville": "SVQ",
        "Rome (Fiumicino)": "FCO", "Milan (Malpensa)": "MXP", "Venice (Marco Polo)": "VCE",
        "Naples": "NAP", "Milan (Linate)": "LIN", "Catania": "CTA"
    }
    return mapping.get(city.strip(), "Unknown")


def get_base_price(airport: str, date: str) -> float:
    print(f"[TOOL] get_base_price('{airport}', '{date}')")
    if airport in (None, "Unknown"): airport = "LHR"
    if len(airport) != 3: airport = get_airport(airport)
    if airport == "Unknown": return "Unknown"

    airport = airport.upper()
    base = 100 if "A" <= airport[0] <= "F" else \
        200 if "G" <= airport[0] <= "L" else \
            300 if "M" <= airport[0] <= "R" else 400

    try:
        d = datetime.strptime(date, "%Y-%m-%d")
    except:
        d = datetime.now()

    if 4 <= d.month <= 9: base *= 1.5  # summer is more expensive
    if d.weekday() >= 5: base *= 1.3  # weekends are also more expensive
    return round(base, 2)


# ------------------------------------------------------------------
#  DISPATCHER
# ------------------------------------------------------------------
def _resolve_airport(args: dict) -> str:
    airport = args.get("airport")
    if airport and len(airport) != 3:
        airport = get_airport(airport)
    return airport


TOOL_HANDLERS = {
    "get_airport": lambda a: {"airport_code": get_airport(a.get("city", ""))},
    "get_base_price": lambda a: {"price": get_base_price(_resolve_airport(a), a.get("date", ""))}
}

# ------------------------------------------------------------------
#  GREETING DETECTION – only block TOOL EXECUTION, not tool visibility
# ------------------------------------------------------------------
GREETING_KEYWORDS = {"hi", "hello", "hey", "good morning", "good afternoon", "how are you", "thanks"}


def is_greeting(msg: str) -> bool:
    lower = msg.lower().strip()
    return any(k in lower for k in GREETING_KEYWORDS) and len(lower.split()) <= 5


# ------------------------------------------------------------------
#  CHAT LOOP – TOOLS ALWAYS PASSED
# ------------------------------------------------------------------
def chat(user_msg: str, history: list):
    # Build messages
    messages = [{"role": "system", "content": SYSTEM_PROMPT}]
    for turn in history:
        messages.append({"role": turn["role"], "content": turn["content"]})
    messages.append({"role": "user", "content": user_msg})

    # ALWAYS pass tools (model decides whether to call them)
    resp = ollama.chat(model=MODEL, messages=messages, tools=TOOLS)
    msg = resp.get("message", {})

    # BLOCK TOOL EXECUTION FOR GREETINGS
    if is_greeting(user_msg):
        return "Hello! How can I help you with flights today?"

    # TOOL CALL?
    if msg.get("tool_calls"):
        call = msg["tool_calls"][0]
        fn = call["function"]["name"]
        args = call["function"]["arguments"]

        payload = TOOL_HANDLERS.get(fn, lambda _: {"error": "unknown"})(args)

        messages.append(msg)
        messages.append({
            "role": "tool",
            "name": fn,
            "content": json.dumps(payload),
            "tool_call_id": call.get("id", "unknown")
        })

        final = ollama.chat(model=MODEL, messages=messages, tools=TOOLS)
        return final["message"]["content"]

    # FALLBACK: malformed JSON in content
    content = msg.get("content", "")
    if "{" in content and "}" in content:
        try:
            start = content.find("{")
            end = content.rfind("}") + 1
            parsed = json.loads(content[start:end])
            if "name" in parsed:
                fn = parsed["name"]
                args = parsed.get("parameters", {}) or parsed.get("arguments", {})
                payload = TOOL_HANDLERS.get(fn, lambda _: {"error": "unknown"})(args)

                messages.append(msg)
                messages.append({
                    "role": "tool",
                    "name": fn,
                    "content": json.dumps(payload),
                    "tool_call_id": "fallback"
                })
                final = ollama.chat(model=MODEL, messages=messages, tools=TOOLS)
                return final["message"]["content"]
        except Exception as e:
            print("JSON parse error:", e)

    return content or "I don't know."


# ------------------------------------------------------------------
#  GRADIO UI
# ------------------------------------------------------------------
gr.ChatInterface(
    fn=chat,
    type="messages",
    title="FlyCheap – Accurate Flight Assistant",
    description="Ask for airport codes or ticket prices. Departure: London Heathrow (LHR)"
).launch()